# HLCV Assignment 1
This notebook contains skeletons that you are supposed to fill in (watch out for <font color='red'>**TODO**</font> tags). In general, you are expected to implement methods yourself. For example, when asked to implement a method computing histograms you can use numpy array functions that help you implement this task, but not use the numpy implementation `np.histogram` for your solution. To submit your assignment, upload your completed notebook and a PDF report (use the provided Latex template with, for example, [Overleaf](http://overleaf.com)) that contains your observations.

In [ ]:
import matplotlib.pyplot as plt, numpy as np, glob, os, scipy, itertools
# you do not need tqdm, but you might find it convenient
try:
    import tqdm.notebook as tqdm
except:
    class _tqdm:
        def tqdm(self, l):
            return l
    tqdm = _tqdm()

In this exercise you will first familiarise yourself with basic image filtering routines. The zip file contains image `graf.png`, which we will use for testing purposes.

# Part 1: Loading and Manipulating Images
We can load and show images using `matplotlib` for now.

In [ ]:
graf = plt.imread("graf.png")
plt.imshow(graf)

An image loaded with matplotlib is a matrix containing brightness values ∈ [0,1] for each color channel.

In [ ]:
graf.shape, graf[0,0,0]

Usually, images are saved in RGB (3 color channels) and uint8 (1 byte = 256 brightness values). In any case, the loaded brightness values are initially discrete.

In [ ]:
graf[0,0,0], graf[0,0,0]*255

In [ ]:
graf_uint8 = (graf*255).astype(np.uint8)

f, ax = plt.subplots(1,2)
ax[0].imshow(
    graf
)
i = ax[1].imshow(
    graf_uint8
)
plt.colorbar(i, fraction=0.04)
graf_uint8[0,0,0]

In [ ]:
img = graf.copy()
img[100:500,100:200,:] = 0
img[250:350,200:300,:] = 0
img[100:500,300:400,:] = 0
img[100:500,500:600,:] = 0
plt.imshow(img)

<font color='red'>**TODO**</font>: Create 3 copies of the `graf` image where each only contains values in one color channel.

In [ ]:
img = graf.copy()
# ... continue here

<font color='red'>**TODO**</font>: Now lets turn the `graf` image into a gray value image by averaging over the color channels. Hint: `plt.imshow` has a default value for the argument `cmap` that you might want to change.

In [ ]:
graf_gray = graf.copy()
# ... continue here

# Part 2: Image Filtering
## a) Gaussian Filter
<font color='red'>**TODO**</font>: Lets start with a simple 1D Gaussian. Implement a method which computes the values of a 1D Gaussian for a given variance σ². The method should also give a vector of values on which the Gaussian filter is defined: integer values on the interval [-3σ, 3σ].

$$
G = \frac{1}{\sqrt{2 \pi \sigma}} \exp\left(-\frac{x^2}{2\sigma^2}\right)
$$

In [ ]:
def gauss(sigma):
    x = np.arange(-int(3*sigma), int(3*sigma)+1)
    # ... continue here
    y = ...
    return x, y

In [ ]:
# plot your Gauss
sigma = 4.0
x, y = gauss(sigma)
plt.plot(x, y)
plt.grid(linestyle="--")

<font color='red'>**TODO**</font>: Implement a 2D Gaussian filter. The function should take an image as an input and return the result of convolution of this image with 2D Gaussian kernel. First, lets start by implementing the 2D Gaussian kernel. Complete function `gausskernel` below.

In [ ]:
def gausskernel(sigma):
    # ... continue here
    return kernel

In [ ]:
# plot your Gauss
kernel = gausskernel(2.0)
plt.imshow(kernel)
plt.colorbar()

<font color='red'>**TODO**</font>: Now implement a convolution operation by completing function `my_convolution`. Afterwards you can test function `gaussfilter` - it should blur our example images.

In [ ]:
def my_padding(image, padding=0):
    if padding != 0:
        imagePadded = np.zeros((image.shape[0] + padding*2, image.shape[1] + padding*2))
        imagePadded[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = image
        return imagePadded
    
    return image

In [ ]:
def my_convolution(image, kernel, padding=0, verbose=True):
    multiplications = 0
    
    kernel = np.flipud(np.fliplr(kernel))
    xKernShape = kernel.shape[0] 
    yKernShape = kernel.shape[1] 
    xImgShape = image.shape[0] 
    yImgShape = image.shape[1]
    xOutput = int((xImgShape - xKernShape + 2 * padding) + 1) 
    yOutput = int((yImgShape - yKernShape + 2 * padding) + 1)
    output = np.zeros((xOutput, yOutput))
    if verbose:
        print(output.shape)
    
    # Apply Equal Padding to All Sides
    imagePadded = my_padding(image, padding)
    
    # Iterate through image
    for y in range(yOutput):
        for x in range(xOutput):
            output[x,y] = # ... continue here
            multiplications += xKernShape*yKernShape
    
    if verbose:
        print(f"Multiplications: {multiplications}")
    return output

def gaussfilter(image, sigma, padding=0):
    kernel = gausskernel(sigma)
    filtered = my_convolution(image, kernel, padding)
    return filtered

Here you can test your implementation. Start with `sigma=4.0` and play around with different sigma values.

In [ ]:
graf_smooth = gaussfilter(graf_gray, sigma=4.0)

plt.imshow(graf_smooth, cmap="gray")
plt.show()

## b) Separability
The Gaussian filter is separable. This means we can first convolve each row of an image with a 1D filter and then convolve each column with another 1D filter instead of applying the corresponding 2D kernel (see lecture slides). Function `gaussfilter_sep` convolves a given image with two 1D Gaussians.

<font color='red'>**TODO**</font>: Test how much more efficient this method is compared to its 2D counterpart. What would you expect?

In [ ]:
def gaussfilter_sep(image, sigma, padding=0):
    _, kernel = gauss(sigma)
    kernel = np.atleast_2d(kernel) / kernel.sum()
    filtered = my_convolution(image, kernel, padding)
    filtered = my_convolution(filtered, kernel.T, 0)
    return filtered

In [ ]:
graf_smooth_sep = gaussfilter_sep(graf_gray, sigma=4.0, padding=0)
plt.imshow(graf_smooth_sep, cmap="gray")
plt.show()

print(f"Diff. graf_smooth vs. graf_smooth_sep: {abs(graf_smooth-graf_smooth_sep).sum()}")

## c) Derivative Filter
<font color='red'>**TODO**</font>: Implement a function `gaussdx` for creating a Gaussian derivative filter in 1D according to the following equation:

$$
\frac{d}{dx} G 
= \frac{d}{dx} \frac{1}{\sqrt{2 \pi \sigma}} \exp\left(-\frac{x^2}{2\sigma^2}\right) \\
= -\frac{1}{\sqrt{2 \pi \sigma^3}} x \exp\left(-\frac{x^2}{2\sigma^2}\right)
$$

In [ ]:
def gaussdx(sigma):
    x = np.arange(-int(3*sigma), int(3*sigma)+1)
    y = # ... continue here
    return x, y

In [ ]:
# plot your derivative filter
sigma = 4.0
x, y = gaussdx(sigma)
plt.plot(x, y)
plt.grid(linestyle="--")

The effect of applying a filter can be studied by observing its so-called impulse response. For this, we create a test image in which only the central pixel has a non-zero value:

In [ ]:
test_img = np.zeros((25,25))
test_img[12,12] = 1
plt.imshow(test_img)
plt.colorbar()

Now, given the following **1D** filter kernels:

In [ ]:
_, G = gauss(1)
G = np.atleast_2d(G).T
_, D = gaussdx(1)
D = np.atleast_2d(D).T

<font color='red'>**TODO**</font>: What happens when you apply the following filter combinations? (G' is the transpose of G)
1. first G, then G'
2. first G, then D'
3. first D, then G'
4. first G', then D
5. first D', then G

First visualize each result on the test image. Then, answer the following question: What is the expected result when applying these filters to images?

In [ ]:
# ... continue here

<font color='red'>**TODO**</font>: Use functions `gauss` and `gaussdx` in order to implement a function `gaussderiv` that returns the 2D Gaussian derivatives of an input image in x and y direction. Try the function on the given example images and comment on the output.

In [ ]:
def gaussderiv(image, sigma, verbose=True):
    # ... continue here
    output1 = my_convolution(image, kernel, 0, verbose=verbose)
    output2 = my_convolution(image, kernel.T, 0, verbose=verbose)
    return kernel, output1, output2

In [ ]:
kernel, dx, dy = gaussderiv(graf_gray, sigma=2.0)

plt.imshow(kernel)
plt.show()
plt.imshow(dx, cmap="gray")
plt.show()
plt.imshow(dy, cmap="gray")
plt.show()

# Part 3: Object Identification
In this part you will develop a simple image querying system which accepts a query image as input and then finds a set of similar images in the database. In order to compare images you will implement several distance functions introduced in the lecture and evaluate their performance in combination with different image representations. The data contains model and query images, which will be used to evaluate your implementation. The model and query images correspond to the same set of objects photographed from different viewpoints.
## Loading Images
The following cell loads all model and query images into numpy arrays. Make sure the *model* and *query* folders are in the same folder as this notebook's file.

In [ ]:
files_model = []
images_model = []
files_query = []
images_query = []

files = [f"obj{i}__0.png" for i in range(1,101)]

for f in files:
    f = os.path.join(".", "model", f)
    if os.path.isfile(f):
        name = os.path.basename(f).replace("__0.png","")
        files_model += [name]
        images_model += [plt.imread(f)]
        file_query = f"{name}__40.png"
        files_query += [file_query]
        images_query += [plt.imread(os.path.join(".", "query", file_query))]

# stack and convert to grayscale
images_model = np.stack(images_model)
images_model_gray = 0.2989 * images_model[:,:,:,0] + \
                    0.5870 * images_model[:,:,:,1] + \
                    0.1140 * images_model[:,:,:,2]
images_query = np.stack(images_query)
images_query_gray = 0.2989 * images_query[:,:,:,0] + \
                    0.5870 * images_query[:,:,:,1] + \
                    0.1140 * images_query[:,:,:,2]

files_model = np.array(files_model)
files_query = np.array(files_query)

print(images_model.shape, images_model_gray.shape)

In [ ]:
f, ax = plt.subplots(1,3)
ax[0].imshow(images_model[0])
ax[1].imshow(images_model_gray[0], cmap="gray")
ax[2].imshow(images_query[0])
ax[0].axis("off")
ax[1].axis("off")
ax[2].axis("off")
plt.show()

## a) Image Representations, Histogram Distances

### Intensity Histogram

Implement function `normalized_hist`, which takes a gray-value image as input and returns the normalized histogram of pixel intensities. Compare results of your implementation with numpy’s `np.histogram` function to check if your implementation works as intended.

<font color='red'>**TODO:**</font> Complete the next cell to compute a normalized histogram of pixel intensities. Do **not** use existing implementations, e.g. `numpy.histogram`, to do so. [Read up about histograms if necessary](https://en.wikipedia.org/wiki/Histogram).

In [ ]:
def normalized_hist(image, nbins=10):
    h = np.zeros(nbins)
    # ... continue here
    return h

Compare your result with numpy's implementation.

In [ ]:
nbins = 10
image_id = 0

h1 = normalized_hist(images_model_gray[image_id], nbins=nbins)
h2, _ = np.histogram(images_model_gray[image_id], bins=np.linspace(0,1,nbins+1))
h2 = h2 / h2.sum()

f = plt.figure(figsize=(5,3))
plt.bar(np.arange(nbins)-0.2, h1, width=0.4)
plt.bar(np.arange(nbins)+0.2, h2, width=0.4)
plt.yscale("log")
plt.xticks(range(nbins))
plt.grid(linestyle="--")
plt.legend(["Yours", "numpy's"])
plt.show()

### RGB Histogram
<font color='red'>**TODO:**</font> Implement the 3-dimensional, normalized RGB-histogram by completing function `rgb_hist`. Go back to the lecture slides for reference.

In [ ]:
def rgb_hist(image, nbins=10):
    h = np.zeros((nbins,nbins,nbins))
    # ... continue here
    return h

Plot your result.

In [ ]:
nbins = 10
image_id = 0

h1 = rgb_hist(images_model[image_id], nbins=nbins)

# lets scale and convert our histogram numbers to log-scale to improve visibility
with np.errstate(divide='ignore'):
    h1 = h1 / min(h1[h1!=0])
    h1 = np.nan_to_num(np.log(h1), neginf=0)*100

idx = np.indices((nbins,nbins,nbins)).reshape(3,-1)

f = plt.figure(figsize=(10,6))

ax = f.add_subplot(1,2,1)
ax.imshow(images_model[image_id])
ax.set_title(files_model[image_id])

ax = f.add_subplot(1,2,2, projection="3d")
ax.scatter3D(
    idx[0], idx[1], idx[2],
    c = [( (idx[0,i]+1)/nbins, (idx[1,i]+1)/nbins, (idx[2,i]+1)/nbins ) for i in range(nbins**3)],
    s = [h1[idx[0,i],idx[1,i],idx[2,i]] for i in range(nbins**3)]
)
ax.set_xticks(range(nbins))
ax.set_yticks(range(nbins))
ax.set_zticks(range(nbins))
ax.set_title("RGB Histogram")
ax.set_xlabel("Red")
ax.set_ylabel("Green")
ax.set_zlabel("Blue")
plt.show()

### RG Histogram
<font color='red'>**TODO:**</font> Complete function `rg_hist` to compute a normalized 2-dimensional chromatic representation RG-histogram (see lecture slides).

In [ ]:
def rg_hist(image, nbins=10):
    h = np.zeros((nbins,nbins))
    # ... continue here
    return h

Plot your result.

In [ ]:
nbins = 10
image_id = 0
h1 = rg_hist(images_model[image_id], nbins=nbins)

# lets scale and convert our histogram numbers to log-scale to improve visibility
with np.errstate(divide='ignore'):
    h1 = h1 / min(h1[h1!=0])
    h1 = np.nan_to_num(np.log(h1), neginf=0)*100

idx = np.array([(r,g,b) for r,g,b in np.indices((nbins,nbins,nbins)).reshape(3,-1).T if r+g+b == nbins]).T

f = plt.figure(figsize=(10,6))

ax = f.add_subplot(1,2,1)
ax.imshow(images_model[image_id])
ax.set_title(files_model[image_id])

ax = f.add_subplot(1,2,2, projection="3d")
ax.scatter3D(
    idx[0], idx[1], idx[2],
    c = [( (idx[0,i]+1)/nbins, (idx[1,i]+1)/nbins, (idx[2,i]+1)/nbins ) for i in range(idx.shape[1])],
    s = [h1[idx[0,i],idx[1,i]] for i in range(idx.shape[1])]
)
ax.set_xticks(range(nbins))
ax.set_yticks(range(nbins))
ax.set_zticks(range(nbins))
ax.set_title("RGB Histogram")
ax.set_xlabel("Red")
ax.set_ylabel("Green")
ax.set_zlabel("Blue")
plt.show()

### dxdy Histogram
<font color='red'>**TODO:**</font> The dxdy histogram computes the 2D histogram of partial derivatives in x and y directions. Implement this histogram by completing function `dxdy_hist`. Hint: Scale the values to [0,1] before computing the histogram.

In [ ]:
def dxdy_hist(image, nbins=10):
    h = np.zeros((nbins,nbins))
    _, dx, dy = gaussderiv(image, sigma=1, verbose=False)
    # ... continue here
    return h

Plot your result.

In [ ]:
nbins = 11
image_id = 0
h1 = dxdy_hist(images_model_gray[image_id], nbins=nbins)

# lets scale and convert our histogram numbers to log-scale to improve visibility
with np.errstate(divide='ignore'):
    h1 = h1 / min(h1[h1!=0])
    h1 = np.nan_to_num(np.log(h1), neginf=0)*100

idx = np.indices((nbins,nbins)).reshape(2,-1)

f = plt.figure(figsize=(10,6))

ax = f.add_subplot(1,2,1)
ax.imshow(images_model_gray[image_id], cmap="gray")
ax.set_title(files_model[image_id])

ax = f.add_subplot(1,2,2)
ax.scatter(
    idx[0], idx[1],
    c = "black",
    s = [h1[idx[0,i],idx[1,i]] for i in range(idx.shape[1])]
)
ax.set_xticks(range(nbins))
ax.set_yticks(range(nbins))
ax.set_title("dxdy Histogram")
ax.set_xlabel("dx")
ax.set_ylabel("dy")
ax.set_aspect("equal")
ax.grid(linestyle="--")
plt.show()

## b) Histogram Comparison Measures
Implement the histogram distance functions discussed during the lecture by filling the missing code in functions `dist_l2`, `dist_intersect`, and `dist_chi2`.
### Euclidean Distance
<font color='red'>**TODO:**</font> Implement the euclidean distance function to compare two histograms *x* and *y*.

In [ ]:
def dist_l2(x,y):
    d = # ... continue here
    return d

Test your implementation:

In [ ]:
a = np.array([1,2,3,4]) / 10
b = np.array([1,2,4,7]) / 15
dist_l2(a,b) == 0.01111111111111111

### Intersection
<font color='red'>**TODO:**</font> Implement the intersection distance function to compare two histograms *x* and *y*.

In [ ]:
def dist_intersect(x,y):
    # compute intersection distance between x and y
    # return 1 - intersection, so that smaller values also correspond to more similar histograms
    intersection = # continue here ...
    return 1-intersection

Test your implementation:

In [ ]:
a = np.array([1,2,3,4]) / 10
b = np.array([1,2,4,7]) / 15
dist_intersect(a,b) == 0.1333333333333333

### Chi-Square
<font color='red'>**TODO:**</font> Implement the chi-square distance function to compare two histograms *x* and *y*. Note: Add a small value (1E-6) to the denominator to avoid division by zero.

In [ ]:
def dist_chi2(x,y):
    # compute chi2 distance between x and y
    # add small value to denominator to not divide by zero
    d = ...
    return d

Test your implementation:

In [ ]:
a = np.array([1,2,3,4]) / 10
b = np.array([1,2,4,7]) / 15
dist_chi2(a,b) == 0.027088900064936177

## c) Object Identification
Having implemented different distance functions and image histograms, we can now test how suitable they are for retrieving images in a query-by-example scenario. Function `find_best_match` takes a list of model images and a list of query images and for each query image returns the index of closest model image according to the given histogram function, distance function, and number of bins. Additionally to the indices of the best matching images, the function returns a matrix which contains distances between all pairs of model and query images.

In [ ]:
def find_best_match(model_ids, query_ids, f_hist, f_dist, bins):
    distances = np.zeros((len(query_ids), len(model_ids)))
    
    m_histograms = []
    for idx in model_ids:
        img = images_model[idx] if f_hist in [rgb_hist, rg_hist] else images_model_gray[idx]
        m_histograms += [f_hist(img, bins)]
    
    q_histograms = []
    for idx in query_ids:
        img = images_query[idx] if f_hist in [rgb_hist, rg_hist] else images_query_gray[idx]
        q_histograms += [f_hist(img, bins)]
    
    for q_i, q_hist in enumerate(q_histograms):
        for m_i, m_hist in enumerate(m_histograms):
            distances[q_i, m_i] = f_dist(q_hist, m_hist)
    
    best_match = np.array(range(len(files_query)))[distances.argmin(axis=0)]
    
    return best_match, distances

In [ ]:
best_match, distances = find_best_match(
    model_ids = range(len(files_model)),
    query_ids = range(len(files_query)),
    f_hist = rgb_hist,
    f_dist = dist_l2,
    bins = 10
)

### Show Neighbors

Implement a function `show_neighbors` which takes a list of model images and a list of query images and for each query image visualizes several model images which are closest to the query image according to the specified distance metric. Use the function `find_best_match` in your implementation.

<font color='red'>**TODO:**</font> Implement a function that plots the closest model images according to given query images and a combination of histogram function, distance function, and number of bins.

In [ ]:
def show_neighbors(model_ids, query_ids, f_hist, f_dist, bins, show_neighbors=5):
    ...

In [ ]:
show_neighbors(
    model_ids = range(len(files_model)),
    query_ids = range(4),
    f_hist = dxdy_hist,
    f_dist = dist_l2,
    bins = 10
)

### Recognition Rate

Use function `find_best_match` to compute recognition rates for different combinations of distance and histogram functions. The recognition rate is given by a ratio between number of correct matches and total number of query images. A match is correct if the best match of a given query image is the corresponding model image, hence, the same object. Example: Given 10 query images and 6 correct matches, the recognition rate is 6/10 = 0.6.

<font color='red'>**TODO:**</font> Compute recognition rates of different combinations of histogram functions, distance functions, and number of bins. Try to find a combination that works best. Why do you think the combination you found works particularly well? Discuss your results.

In [ ]:
hist_types = [normalized_hist, rgb_hist, rg_hist, dxdy_hist]
dist_types = [dist_l2, dist_intersect, dist_chi2]
bins_list = # your choice

# continue here ...